In [ ]:
import cv2
import mediapipe as mp
import numpy as np
from datetime import datetime
from pytube import YouTube

In [ ]:
###  自分の動画 ###
# ビデオのパスを直接指定
video_path = r"C:\Users\Hassan\Desktop\ボルダリング\input\IMG_923512.mov"

### youtube ###

# def download_youtube_video(url, path):
#     try:
#         yt = YouTube(url)
#         video = yt.streams.filter(file_extension='mp4').first()
#         video.download(output_path=path, filename=yt.title + '.mp4')
#         return yt.title + '.mp4'
#     except Exception as e:
#         print("Error downloading video:", e)
#         return None
# # YouTube URL
# youtube_url = "https://www.youtube.com/shorts/Ag9YoMdG4yM"

In [ ]:
# モジュールの初期化
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 日付を取得し、フォーマットする
today_date = datetime.now().strftime("%Y%m%d")
print("Today's Date:", today_date)  # デバッグ出力

# 元ファイル名を取得し、新しいファイル名を生成
base_name = video_path.split("\\")[-1].split(".")[0]
output_name = f"{base_name}_{today_date}.mov"
print("Output File Name:", output_name)  # デバッグ出力

# 各関節のマーカーの色を白で定義
white_color = (255, 255, 255)

# VideoWriterの設定
# VideoWriterの設定
cap = cv2.VideoCapture(video_path)

# 元の動画のフレームレートを取得
original_fps = cap.get(cv2.CAP_PROP_FPS)

# 元の動画のフレームレートを使用してVideoWriterを設定
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_name, fourcc, original_fps, (int(cap.get(3)), int(cap.get(4))))


In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)

    if results.pose_landmarks:
        # 関節間の接続線を描画（顔のランドマークを除外）
        for connection in mp_pose.POSE_CONNECTIONS:
            start, end = connection
            # 顔と口のランドマークを除外
            if start not in [mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.RIGHT_EYE_INNER, mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.RIGHT_EYE_OUTER, mp_pose.PoseLandmark.LEFT_EYE_INNER, mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.LEFT_EYE_OUTER, mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.MOUTH_LEFT, mp_pose.PoseLandmark.MOUTH_RIGHT] and \
               end not in [mp_pose.PoseLandmark.NOSE, mp_pose.PoseLandmark.RIGHT_EYE_INNER, mp_pose.PoseLandmark.RIGHT_EYE, mp_pose.PoseLandmark.RIGHT_EYE_OUTER, mp_pose.PoseLandmark.LEFT_EYE_INNER, mp_pose.PoseLandmark.LEFT_EYE, mp_pose.PoseLandmark.LEFT_EYE_OUTER, mp_pose.PoseLandmark.RIGHT_EAR, mp_pose.PoseLandmark.LEFT_EAR, mp_pose.PoseLandmark.MOUTH_LEFT, mp_pose.PoseLandmark.MOUTH_RIGHT]:
                cv2.line(frame, 
                         (int(results.pose_landmarks.landmark[start].x * frame.shape[1]), int(results.pose_landmarks.landmark[start].y * frame.shape[0])),
                         (int(results.pose_landmarks.landmark[end].x * frame.shape[1]), int(results.pose_landmarks.landmark[end].y * frame.shape[0])),
                         white_color, 2) # 線の太さ
                
        # 各関節にマーカーを配置
        for id, landmark in enumerate(results.pose_landmarks.landmark):
            landmark_type = mp_pose.PoseLandmark(id).name
            if landmark_type in ["LEFT_SHOULDER", "RIGHT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW", 
                                 "LEFT_WRIST", "RIGHT_WRIST", "LEFT_HIP", "RIGHT_HIP", 
                                 "LEFT_KNEE", "RIGHT_KNEE", "LEFT_ANKLE", "RIGHT_ANKLE"]:
                cv2.circle(frame, (int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])), 7, white_color, -1) # マーカーサイズ

        # 腰の中心位置の計算
        left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
        hip_center_x = int((left_hip.x + right_hip.x) * 0.5 * frame.shape[1])
        hip_center_y = int((left_hip.y + right_hip.y) * 0.5 * frame.shape[0])

        # 肩の中心位置の計算
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        shoulder_center_x = int((left_shoulder.x + right_shoulder.x) * 0.5 * frame.shape[1])
        shoulder_center_y = int((left_shoulder.y + right_shoulder.y) * 0.5 * frame.shape[0])

        # 背骨（肩の中心から腰の中心まで）の描画
        cv2.line(frame, (shoulder_center_x, shoulder_center_y), (hip_center_x, hip_center_y), white_color, 2)# 線の太さ

        # 逆三角形を腰の中心位置に描画
        triangle_size = 20
        triangle_points = np.array([
            (hip_center_x, hip_center_y + triangle_size),  # 下部の点
            (hip_center_x - triangle_size, hip_center_y - triangle_size),  # 左上の点
            (hip_center_x + triangle_size, hip_center_y - triangle_size)  # 右上の点
        ])
        cv2.drawContours(frame, [triangle_points], 0, white_color, -1)

    # フレームを出力
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()